#**FairLearn Code Snippet**

Fairlearn is an open-source, Python package that aims to help data scientists improve fairness in machine learning models. It provides algorithms and tools to assess and mitigate unfairness in models. While Fairlearn does not offer a specific "Fairlearn Logistic Regression Classifier" out of the box, you can use Fairlearn's tools alongside scikit-learn's LogisticRegression to assess and mitigate bias.

Here's a simple example to demonstrate how you can use Fairlearn with a logistic regression classifier from scikit-learn. This example will involve the following steps:
- Load a dataset and split it into features and target.
- Train a standard logistic regression model.
- Use Fairlearn's MetricFrame to assess fairness-related metrics on the model's predictions.
- Apply a mitigation technique from Fairlearn to reduce bias.

In [ ]:
pip install scikit-learn fairlearn


In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from fairlearn.metrics import MetricFrame, selection_rate
from sklearn.metrics import accuracy_score
from fairlearn.postprocessing import ThresholdOptimizer
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load a dataset
X, y = fetch_openml(data_id=1590, as_frame=True, return_X_y=True)  # Adult census dataset
y = (y == '>50K').astype(int)  # Binary encoding of the target

# Assume the sensitive feature is one of the columns, e.g., 'sex' or 'race'. Adjust accordingly.
sensitive_feature_name = 'sex'

# Ensure the sensitive feature is included before scaling
sensitive_feature = X[[sensitive_feature_name]]

# Preprocessing
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X.select_dtypes(include=['float64', 'int64'])), columns=X.select_dtypes(include=['float64', 'int64']).columns)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test, sensitive_feature_train, sensitive_feature_test = train_test_split(X_scaled, y, sensitive_feature, test_size=0.3, random_state=42)

# Train a logistic regression model
lr = LogisticRegression(max_iter=10000)
lr.fit(X_train, y_train)

# Predictions
y_pred = lr.predict(X_test)

# Assess fairness using MetricFrame
print('Original Metrics BEFORE Bias Mitigation\n')
mf = MetricFrame(metrics={'accuracy': accuracy_score, 'selection_rate': selection_rate},
                 y_true=y_test,
                 y_pred=y_pred,
                 sensitive_features=sensitive_feature_test[sensitive_feature_name])

print(mf.by_group)

# Mitigate bias using ThresholdOptimizer
postprocessor = ThresholdOptimizer(estimator=lr, constraints="equalized_odds", prefit=True)
postprocessor.fit(X_train, y_train, sensitive_features=sensitive_feature_train[sensitive_feature_name])

# Adjusted predictions
y_pred_adj = postprocessor.predict(X_test, sensitive_features=sensitive_feature_test[sensitive_feature_name])

# Re-evaluate fairness
print('Original Metrics AFTER Bias Mitigation\n')
mf_adj = MetricFrame(metrics={'accuracy': accuracy_score, 'selection_rate': selection_rate},
                     y_true=y_test,
                     y_pred=y_pred_adj,
                     sensitive_features=sensitive_feature_test[sensitive_feature_name])

print(mf_adj.by_group)


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Original Metrics BEFORE Bias Mitigation

        accuracy  selection_rate
sex                             
Female  0.897959        0.071325
Male    0.780249        0.160477


/usr/local/lib/python3.10/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:288: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  warn(


Original Metrics AFTER Bias Mitigation

        accuracy  selection_rate
sex                             
Female  0.891569         0.08019
Male    0.776576         0.15150
